# Refazendo imports e manipulação dos dados

In [32]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


dataset = pd.read_csv('db/movie_dataset.csv', sep=',',  index_col=0, na_values='?')

dataset = dataset.loc[:, ~dataset.columns.isin(['original_title', 'overview', 'tagline', 'title', 'homepage', 'crew', 'keywords', 'original_language', 'id'])]
dataset = dataset.loc[dataset['status'] == 'Released']

dataset_base = dataset.loc[:, dataset.columns.isin(['budget', 'revenue', 'popularity', 'runtime', 'vote_average', 'vote_count'])]
noise = np.random.randint(0, 101, size=dataset_base['revenue'].shape[0])
dataset_base['revenue'] = dataset_base['revenue'] + noise

dataset_base_rev_class = pd.qcut(dataset_base['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_base['revenue'] = dataset_base_rev_class

dataset_dummies = dataset
dataset_dummies = pd.get_dummies(dataset_dummies)
dataset_dummies['revenue'] = dataset_dummies['revenue'] +noise
dataset_dummies_rev = pd.qcut(dataset_dummies['revenue'], q=4, labels=['very low', 'low', 'medium', 'high'])
dataset_dummies['revenue'] = dataset_dummies_rev

/tmp/ipykernel_36739/2331555734.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base['revenue'] + noise
/tmp/ipykernel_36739/2331555734.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_base['revenue'] = dataset_base_rev_class


# Naive Bayes

In [33]:
dataset_base = dataset_base.loc[:, dataset_base.columns != 'vote_average']
dataset_base = dataset_base.loc[:, dataset_base.columns != 'runtime']

rev_pred_paramX = dataset_base.loc[:, dataset_base.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_base.loc[:, dataset_base.columns == 'revenue']).ravel()

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

gnb = GaussianNB()
pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != pred).sum()))
acc = gnb.score(X_test, y_test)
print(f'Accuracy: {acc :.2%}')

Number of mislabeled points out of a total 480 points : 258
Accuracy: 46.25%


In [34]:
rev_pred_paramX = dataset_dummies.loc[:, dataset_dummies.columns != 'revenue'] 
rev_pred_paramY = np.array(dataset_dummies.loc[:, dataset_dummies.columns == 'revenue']).ravel()

rev_pred_paramX.fillna(1, inplace=True)
rev_pred_paramY = np.nan_to_num(rev_pred_paramY, nan=1)

X_train, X_test, y_train, y_test = train_test_split(rev_pred_paramX, rev_pred_paramY, test_size=0.10, stratify=rev_pred_paramY, random_state=42)

gnb = GaussianNB()
pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != pred).sum()))
acc = gnb.score(X_test, y_test)
print(f'Accuracy: {acc :.2%}')

/tmp/ipykernel_36739/938934480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_pred_paramX.fillna(1, inplace=True)


Number of mislabeled points out of a total 480 points : 258
Accuracy: 46.25%
